In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from scipy.stats import lognorm, expon, kstest
from pathlib import Path
from ab_testing.constants import DISTRIBUTIONS, client_name, target_col

from fitter import Fitter, get_distributions, get_common_distributions

df = pd.read_parquet(Path("raw_data") / f"{client_name}_data.p")
data = df[df[target_col] > 0]

sns.set_style('white')
sns.set_context("paper", font_scale = 2)
sns.displot(data=data, x=target_col, kind="hist", bins = 100, aspect = 1.5)

AIC and BIC for (common) distributions

In [ ]:
df = pd.DataFrame(columns=["distribution", "AIC", "BIC"])
dists = []
aic = []
bic = []
for com_dist in get_common_distributions():
    dist = eval("scipy.stats." + com_dist)
    params = dist.fit(data[target_col].values)
    pdf_fitted = dist.pdf(data[target_col].values, *params)
    
    logLik = np.sum(dist.logpdf(data[target_col].values, *params))
    k = len(params[:])
    n = len(data[target_col].values)
    dists.append(com_dist)
    aic.append(2 * k - 2 * logLik)
    bic.append(k * np.log(n) - 2 * logLik)

df["distribution"] = dists
df["AIC"] = aic
df["BIC"] = bic
df.sort_values(by="AIC", inplace=True)
df.reset_index(inplace=True, drop=True)
df

Graphical fit for exponencial and lognormal distributions

In [ ]:
x = np.arange(len(data))
y = data[target_col].values
h = plt.hist(y, bins=range(48))

for dist_name in DISTRIBUTIONS:
    dist = getattr(scipy.stats, dist_name)
    params = dist.fit(y)
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]
    if arg:
        pdf_fitted = dist.pdf(x, *arg, loc=loc, scale=scale) * len(data)
    else:
        pdf_fitted = dist.pdf(x, loc=loc, scale=scale) * len(data)
    plt.plot(pdf_fitted, label=dist_name)
    plt.xlim(0, np.quantile(data[target_col], 0.99))
plt.legend(loc='upper right')
plt.show()

Kolmogorov-SMirnov test for exponencial and lognormal distributions

In [ ]:
print("KS Test:")

print("Exponential distribution:")

loc, scale = expon.fit(data[target_col].values)

stat, p = kstest(data[target_col].values, 'expon', args=(loc, scale))

print("stat    = %9.5f" % stat)
print("p-value = %9.5f" % p)

print("Log normal distribution:")

sigma, loc, scale = lognorm.fit(data[target_col].values)

stat, p = kstest(data[target_col].values, 'lognorm', args=(sigma, loc, scale))

print("stat    = %9.5f" % stat)
print("p-value = %9.5f" % p)

In [ ]:
# https://github.com/tcassou/babtest
# from babtest import BABTest

# df_P = data.loc[data["test_group"] == "P"]
# df_C = data.loc[data["test_group"] == "C"]

# bt = BABTest(df_C[target_col].values, df_P[target_col].values, model='lognormal')
# bt.run()
# bt.plot()